In [38]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import optimizers

from tensorflow import keras

In [3]:
tf.__version__

'2.12.0'

### 선형모형

In [28]:
# data
x = [1, 2, 3, 4, 5, 6, 7, 8, 9] # 공부하는 시간
y = [11, 22, 33, 44, 53, 66, 77, 87, 95] # 각 공부하는 시간에 맵핑되는 성적

model1 = Sequential()
model1.add(Dense(1, input_dim=1, activation='linear'))

sgd = optimizers.SGD(learning_rate=0.01)
model1.compile(optimizer=sgd, loss='mse', metrics=['mse'])

history1 =model1.fit(x, y, epochs=3)

Epoch 1/3
1/1 [==============================] - 1s 605ms/step - loss: 4732.9604 - mse: 4732.9604
Epoch 2/3
1/1 [==============================] - 0s 17ms/step - loss: 583.5018 - mse: 583.5018
Epoch 3/3
1/1 [==============================] - 0s 16ms/step - loss: 72.9403 - mse: 72.9403


In [29]:
model1.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [54]:
X = np.array([[70,85,11], [71,89,18], [50,80,20], [99,20,10], [50,10,10]]) 
y = np.array([73, 82 ,72, 57, 34]) # 최종 성적

model3 = Sequential()
model3.add(Dense(1, input_dim=3, activation='linear'))
sgd = optimizers.SGD(learning_rate=0.0001)
model3.compile(optimizer=sgd, loss='mse', metrics=['mse'])
history3 = model3.fit(X, y, epochs=5, verbose=1)

Epoch 1/5
1/1 [==============================] - 1s 563ms/step - loss: 1873.9705 - mse: 1873.9705
Epoch 2/5
1/1 [==============================] - 0s 12ms/step - loss: 1097.2332 - mse: 1097.2332
Epoch 3/5
1/1 [==============================] - 0s 12ms/step - loss: 657.0801 - mse: 657.0801
Epoch 4/5
1/1 [==============================] - 0s 11ms/step - loss: 404.5827 - mse: 404.5827
Epoch 5/5
1/1 [==============================] - 0s 12ms/step - loss: 257.8143 - mse: 257.8143


In [36]:
model3.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 1)                 4         
                                                                 
Total params: 4
Trainable params: 4
Non-trainable params: 0
_________________________________________________________________


### 로지스틱 회귀분석

In [23]:

x = np.array([-50, -40, -30, -20, -10, -5, 0, 5, 10, 20, 30, 40, 50])
y = np.array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]) # 숫자 10부터 1

model2 = Sequential()
model2.add(Dense(1, input_dim=1, activation='sigmoid'))

sgd = optimizers.SGD(learning_rate=0.01)
model2.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['binary_accuracy'])

history2 = model2.fit(x, y, epochs=3)

Epoch 1/3


1/1 [==============================] - 1s 676ms/step - loss: 0.4306 - binary_accuracy: 0.9231
Epoch 2/3
1/1 [==============================] - 0s 10ms/step - loss: 0.4290 - binary_accuracy: 0.9231
Epoch 3/3
1/1 [==============================] - 0s 18ms/step - loss: 0.4275 - binary_accuracy: 0.9231


In [25]:
model2.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 1)                 2         
                                                                 
Total params: 2
Trainable params: 2
Non-trainable params: 0
_________________________________________________________________


In [31]:
X = np.array([[0, 0], [0, 1], [1, 0], [0, 2], [1, 1], [2, 0]])
y = np.array([0, 0, 0, 1, 1, 1])

model4 = Sequential()
model4.add(Dense(1, input_dim=2, activation='sigmoid'))
model4.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['binary_accuracy'])

history4 = model4.fit(X, y, epochs=3)

Epoch 1/3
1/1 [==============================] - 1s 934ms/step - loss: 0.8129 - binary_accuracy: 0.5000
Epoch 2/3
1/1 [==============================] - 0s 14ms/step - loss: 0.8116 - binary_accuracy: 0.3333
Epoch 3/3
1/1 [==============================] - 0s 12ms/step - loss: 0.8102 - binary_accuracy: 0.3333


In [32]:
model4.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 1)                 3         
                                                                 
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [45]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [46]:
import numpy as np

# Construct and compile an instance of CustomModel
inputs = keras.Input(shape=(32,))
outputs = keras.layers.Dense(1)(inputs)
model = CustomModel(inputs, outputs)
model.compile(optimizer="adam", loss="mse", metrics=["mae"])

# Just use `fit` as usual
x = np.random.random((1000, 32))
y = np.random.random((1000, 1))
model.fit(x, y, epochs=3)

Epoch 1/3
32/32 [==============================] - 1s 3ms/step - loss: 1.6372 - mae: 1.1747
Epoch 2/3
32/32 [==============================] - 0s 3ms/step - loss: 0.7351 - mae: 0.7289
Epoch 3/3
32/32 [==============================] - 0s 2ms/step - loss: 0.3720 - mae: 0.4884


In [41]:
model.summary()

Model: "custom_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32)]              0         
                                                                 
 dense_13 (Dense)            (None, 1)                 33        
                                                                 
Total params: 33
Trainable params: 33
Non-trainable params: 0
_________________________________________________________________


In [49]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [51]:
# %tensorboard --logdir logs/fit